# Modelo Preditivo - Tempo de Entrega (ETA)
Notebook para criar modelo preditivo que melhore a estimativa de tempo de entrega.

**Objetivo:** Melhorar a precisão da estimativa de tempo de entrega (`eta_minutes_quote`) apenas para pedidos dos canais próprios (site_proprio + whatsapp).

**Metodologia:** Divisão train/test dupla, cross-validation e comparação de 5 modelos usando RMSE.


## 1. Setup e Imports


In [258]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')

# Configurações visuais
plt.rcParams["figure.figsize"] = (12, 8)
plt.rcParams["axes.grid"] = True
plt.rcParams["font.size"] = 10

print("Bibliotecas importadas com sucesso!")


Bibliotecas importadas com sucesso!


## 2. Carregamento dos Dados


In [259]:
# Carregar dados limpos
df = pd.read_csv("../tratamento_inicial/Base_Kaiserhaus_Limpa.csv")

print("Dados carregados:")
print(f"   • Total de registros: {len(df):,}")
print(f"   • Colunas: {df.shape[1]}")
print(f"   • Valores nulos: {df.isnull().sum().sum()}")

print("\nColunas disponíveis:")
print(list(df.columns))


Dados carregados:
   • Total de registros: 5,000
   • Colunas: 16
   • Valores nulos: 0

Colunas disponíveis:
['macro_bairro', 'nome_cliente', 'bairro_destino', 'order_datetime', 'platform', 'order_mode', 'distance_km', 'tempo_preparo_minutos', 'status', 'eta_minutes_quote', 'actual_delivery_minutes', 'total_brl', 'classe_pedido', 'platform_commission_pct', 'num_itens', 'satisfacao_nivel']


## 3. Filtro para Canais Próprios


In [260]:
# Filtrar apenas canais próprios
df_canais_proprios = df.copy()



## 4. Feature Engineering e One-Hot Encoding


In [261]:
# Função para obter período do dia
def obter_periodo_dia(hora):
    if 0 <= hora < 6:
        return 'Madrugada'
    elif 6 <= hora < 12:
        return 'Manhã'
    elif 12 <= hora < 18:
        return 'Tarde'
    else:
        return 'Noite'

# Criar período do dia diretamente no DataFrame
df_canais_proprios['periodo_dia'] = pd.to_datetime(df_canais_proprios['order_datetime']).dt.hour.apply(obter_periodo_dia)

# Selecionar features
features_numericas = ['distance_km', 'tempo_preparo_minutos', 'num_itens']
features_categoricas = ['periodo_dia', 'platform']
target = 'actual_delivery_minutes'

# Criar DataFrames separados
X_numericas = df_canais_proprios[features_numericas].copy()
X_categoricas = df_canais_proprios[features_categoricas].copy()
y = df_canais_proprios[target].copy()

# Aplicar One-Hot Encoding nas colunas categóricas SEM remover nenhuma categoria
encoder = OneHotEncoder(sparse_output=False, drop=None)
X_categoricas_encoded = encoder.fit_transform(X_categoricas)

# Criar DataFrame com nomes das colunas
feature_names = encoder.get_feature_names_out(features_categoricas)
X_categoricas_df = pd.DataFrame(X_categoricas_encoded, columns=feature_names, index=X_categoricas.index)

# Combinar features numéricas e categóricas
X_final = pd.concat([X_numericas, X_categoricas_df], axis=1)

# Salvar base com One-Hot Encoding aplicado
df_base_final = pd.concat([df_canais_proprios[features_numericas + ['actual_delivery_minutes']], X_categoricas_df], axis=1)
df_base_final.to_csv("base_modelo_preditivo_eta.csv", index=False)

print("FEATURE ENGINEERING COMPLETO")
print("=" * 50)
print("Arquivo: 'base_modelo_preditivo_eta.csv'")
print(f"Total de registros: {len(df_base_final):,}")
print(f"Colunas: {df_base_final.shape[1]}")
print(f"Features categóricas criadas: {len(feature_names)}")

print("\nDistribuição por período do dia:")
distribuicao_periodo = df_canais_proprios['periodo_dia'].value_counts()
for periodo, count in distribuicao_periodo.items():
    pct = count / len(df_canais_proprios) * 100
    print(f"   • {periodo}: {count:,} pedidos ({pct:.1f}%)")



FEATURE ENGINEERING COMPLETO
Arquivo: 'base_modelo_preditivo_eta.csv'
Total de registros: 5,000
Colunas: 12
Features categóricas criadas: 8

Distribuição por período do dia:
   • Tarde: 1,897 pedidos (37.9%)
   • Noite: 1,569 pedidos (31.4%)
   • Manhã: 1,399 pedidos (28.0%)
   • Madrugada: 135 pedidos (2.7%)


## 5. Divisão Train/Test Dupla e Modelos


In [262]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error
import numpy as np

# 1️⃣ Split único (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X_final, y, test_size=0.2, random_state=42, shuffle=True
)

# 2️⃣ Definir modelos
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.dummy import DummyRegressor
from sklearn.ensemble import RandomForestRegressor

modelos = {
    # Dummy Regressor - variações
    'Dummy Regressor (Mean)': DummyRegressor(strategy='mean'),
    
    # Linear Regression - variações
    'Linear Regression (Pipeline)': Pipeline([
        ('scaler', StandardScaler()),
        ('regressor', LinearRegression())
    ]),
    
    # Decision Tree - variações
    'Decision Tree (Default)': DecisionTreeRegressor(random_state=42),
    'Decision Tree (Pruned)': DecisionTreeRegressor(
        max_depth=10, 
        min_samples_split=20, 
        min_samples_leaf=10, 
        random_state=42
    ),
    'Decision Tree (Deep)': DecisionTreeRegressor(
        max_depth=15, 
        min_samples_split=5, 
        min_samples_leaf=2, 
        random_state=42
    ),
    
    # Random Forest - variações
    'Random Forest (100)': RandomForestRegressor(
        n_estimators=100,
        max_depth=10,
        min_samples_split=10,
        min_samples_leaf=5,
        random_state=42
    ),
    'Random Forest (200)': RandomForestRegressor(
        n_estimators=200,
        max_depth=10,
        min_samples_split=10,
        min_samples_leaf=5,
        random_state=42
    ),
    'Random Forest (500)': RandomForestRegressor(
        n_estimators=500,
        max_depth=10,
        min_samples_split=10,
        min_samples_leaf=5,
        random_state=42
    ),
     
    
}

# 3️⃣ Avaliar cada modelo com RMSE de treino e teste
resultados = {}

print("RESULTADOS DE TREINO E TESTE (RMSE)")
print("=" * 55)

for nome, modelo in modelos.items():
    modelo.fit(X_train, y_train)
    
    # Previsões
    y_pred_train = modelo.predict(X_train)
    y_pred_test = modelo.predict(X_test)
    
    # Cálculo de RMSE
    rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
    rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
    
    resultados[nome] = {
        'RMSE Treino': rmse_train,
        'RMSE Teste': rmse_test,
    }
    
    print(f"{nome:25s} → RMSE Treino = {rmse_train:.3f} | RMSE Teste = {rmse_test:.3f}")



RESULTADOS DE TREINO E TESTE (RMSE)
Dummy Regressor (Mean)    → RMSE Treino = 12.632 | RMSE Teste = 12.739
Linear Regression (Pipeline) → RMSE Treino = 5.508 | RMSE Teste = 5.525
Decision Tree (Default)   → RMSE Treino = 0.498 | RMSE Teste = 7.571
Decision Tree (Pruned)    → RMSE Treino = 5.019 | RMSE Teste = 6.031
Decision Tree (Deep)      → RMSE Treino = 3.155 | RMSE Teste = 7.083
Random Forest (100)       → RMSE Treino = 4.565 | RMSE Teste = 5.753
Random Forest (200)       → RMSE Treino = 4.560 | RMSE Teste = 5.749
Random Forest (500)       → RMSE Treino = 4.559 | RMSE Teste = 5.745


In [263]:
# COMPARAÇÃO ETA ANTIGO vs NOVO vs ACTUAL
print(" COMPARAÇÃO ETA ANTIGO vs NOVO vs ACTUAL")
print("=" * 60)

# Usar o melhor modelo (Random Forest 500)
melhor_modelo = modelos['Random Forest (500)']
melhor_modelo.fit(X_final, y)

# Fazer predições
eta_novo = melhor_modelo.predict(X_final)

# Criar DataFrame de comparação
comparacao = pd.DataFrame({
    'Pedido': range(1, len(df_canais_proprios) + 1),
    'ETA_Antigo': df_canais_proprios['eta_minutes_quote'].values,
    'ETA_Novo': eta_novo,
    'Actual_Delivery': df_canais_proprios['actual_delivery_minutes'].values
})

# Calcular diferenças
comparacao['Diff_Antigo'] = comparacao['Actual_Delivery'] - comparacao['ETA_Antigo']
comparacao['Diff_Novo'] = comparacao['Actual_Delivery'] - comparacao['ETA_Novo']

# Calcular erros absolutos
comparacao['Erro_Abs_Antigo'] = abs(comparacao['Diff_Antigo'])
comparacao['Erro_Abs_Novo'] = abs(comparacao['Diff_Novo'])

# Estatísticas gerais
print(f"\n ESTATÍSTICAS GERAIS:")
print(f"   • RMSE ETA Antigo: {np.sqrt(mean_squared_error(comparacao['Actual_Delivery'], comparacao['ETA_Antigo'])):.3f}")
print(f"   • RMSE ETA Novo: {np.sqrt(mean_squared_error(comparacao['Actual_Delivery'], comparacao['ETA_Novo'])):.3f}")
print(f"   • Melhoria: {((np.sqrt(mean_squared_error(comparacao['Actual_Delivery'], comparacao['ETA_Antigo'])) - np.sqrt(mean_squared_error(comparacao['Actual_Delivery'], comparacao['ETA_Novo']))) / np.sqrt(mean_squared_error(comparacao['Actual_Delivery'], comparacao['ETA_Antigo'])) * 100):.1f}%")

# Mostrar alguns exemplos de melhoria
print(f"\n EXEMPLOS DE MELHORIA:")
melhorias = comparacao[comparacao['Erro_Abs_Novo'] < comparacao['Erro_Abs_Antigo']].head(10)
print(f"{'Pedido':<8} {'ETA Antigo':<12} {'ETA Novo':<12} {'Actual':<12} {'Erro Antigo':<12} {'Erro Novo':<12}")
print("-" * 80)
for _, row in melhorias.iterrows():
    print(f"{row['Pedido']:<8} {row['ETA_Antigo']:<12.1f} {row['ETA_Novo']:<12.1f} {row['Actual_Delivery']:<12.1f} {row['Erro_Abs_Antigo']:<12.1f} {row['Erro_Abs_Novo']:<12.1f}")

# Mostrar alguns exemplos de piora
print(f"\n EXEMPLOS DE PIORA:")
pioras = comparacao[comparacao['Erro_Abs_Novo'] > comparacao['Erro_Abs_Antigo']].head(10)
print(f"{'Pedido':<8} {'ETA Antigo':<12} {'ETA Novo':<12} {'Actual':<12} {'Erro Antigo':<12} {'Erro Novo':<12}")
print("-" * 80)
for _, row in pioras.iterrows():
    print(f"{row['Pedido']:<8} {row['ETA_Antigo']:<12.1f} {row['ETA_Novo']:<12.1f} {row['Actual_Delivery']:<12.1f} {row['Erro_Abs_Antigo']:<12.1f} {row['Erro_Abs_Novo']:<12.1f}")

# Contar melhorias vs pioras
melhorias_count = len(comparacao[comparacao['Erro_Abs_Novo'] < comparacao['Erro_Abs_Antigo']])
pioras_count = len(comparacao[comparacao['Erro_Abs_Novo'] > comparacao['Erro_Abs_Antigo']])
iguais_count = len(comparacao[comparacao['Erro_Abs_Novo'] == comparacao['Erro_Abs_Antigo']])

print(f"\n RESUMO:")
print(f"   • Pedidos com melhoria: {melhorias_count} ({melhorias_count/len(comparacao)*100:.1f}%)")
print(f"   • Pedidos com piora: {pioras_count} ({pioras_count/len(comparacao)*100:.1f}%)")
print(f"   • Pedidos iguais: {iguais_count} ({iguais_count/len(comparacao)*100:.1f}%)")

 COMPARAÇÃO ETA ANTIGO vs NOVO vs ACTUAL

 ESTATÍSTICAS GERAIS:
   • RMSE ETA Antigo: 12.179
   • RMSE ETA Novo: 4.598
   • Melhoria: 62.2%

 EXEMPLOS DE MELHORIA:
Pedido   ETA Antigo   ETA Novo     Actual       Erro Antigo  Erro Novo   
--------------------------------------------------------------------------------
1.0      50.0         63.2         62.4         12.4         0.8         
2.0      45.0         37.4         35.6         9.4          1.8         
3.0      43.0         32.1         34.5         8.5          2.4         
4.0      19.0         12.0         14.4         4.6          2.4         
6.0      44.0         35.1         33.5         10.5         1.6         
8.0      40.0         31.7         34.5         5.5          2.8         
9.0      29.0         21.0         19.9         9.1          1.1         
11.0     29.0         12.7         7.2          21.8         5.5         
12.0     47.0         41.4         38.2         8.8          3.2         
13.0     49.0  